In [ ]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [ ]:
from db_client import RDSClient

In [ ]:
db = RDSClient()


In [ ]:
# 수집할 카테고리 (보여지는 텍스트)
CATEGORIES = [
    "영캐주얼",
    "베이직/SPA",
    "스포티/액티브",
    "트래디셔널/아메카지",
    "컨템포러리포멀",
    "유니크디자이너",
    "스트릿캐주얼"
]

MAIN_URL = "https://www.musinsa.com/main/musinsa/recommend?gf=M&tabId=brand"
OUTPUT_CSV = "무신사_전체_브랜드_tags.csv"

def setup_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_window_size(1200, 1000)
    return driver

def close_popups(driver):
    """가벼운 팝업 닫기 시도 (여러 경우 대응)"""
    selectors = [
        'button[aria-label="닫기"]',
        'button[aria-label="close"]',
        'button[class*="JoyrideMDSTooltip__CloseButton"]',
        'button[class*="close"]'
    ]
    for sel in selectors:
        try:
            elems = driver.find_elements(By.CSS_SELECTOR, sel)
            for e in elems:
                try:
                    if e.is_displayed():
                        e.click()
                        time.sleep(0.3)
                except:
                    pass
        except:
            pass

def find_scroll_container(driver):
    """
    브랜드 리스트를 포함하는 스크롤 컨테이너를 찾아 반환.
    실패 시 None 반환(대신 전체 window 스크롤 사용).
    """
    candidates = [
        '[class*="BrandMenu__VirtualContainer"]',
        '[class*="BrandMenu__Wrapper"]',
        '[class*="brand-list"]'
    ]
    for c in candidates:
        try:
            el = driver.find_element(By.CSS_SELECTOR, c)
            # 일부 구조는 내부 element가 실제 스크롤 대상일 수 있으므로 부모로 올려본다
            try:
                parent = el.find_element(By.XPATH, '..')
                return parent
            except:
                return el
        except:
            continue
    return None

def scroll_to_top(driver, container):
    try:
        if container:
            driver.execute_script("arguments[0].scrollTop = 0;", container)
        else:
            driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.6)
    except:
        time.sleep(0.6)

def click_category_tab(driver, category_text):
    """
    화면의 카테고리 탭(텍스트)을 찾아 클릭.
    반환: True (클릭 성공 or 이미 선택), False (찾기/클릭 실패)
    """
    try:
        # 탭 요소는 span 등으로 존재. 텍스트 포함으로 찾는다.
        xpath = f"//span[contains(text(), \"{category_text}\") and (contains(@class,'TabOutlinedItem') or contains(@class,'tab') or contains(@class,'px-1.5'))]"
        tab = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        # 스크롤해서 보이게 하고 클릭
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(0.3)
        tab.click()
        time.sleep(1.2)  # 데이터 로드 대기
        return True
    except TimeoutException:
        # 탭 못찾음
        print(f"✗ '{category_text}' 탭을 찾지 못했습니다.")
        return False
    except Exception as e:
        print(f"✗ '{category_text}' 탭 클릭 중 예외: {e}")
        return False

def collect_brands_from_category(driver, category_name, container=None):
    """
    주어진 카테고리(화면상 선택된 상태)에서 스크롤을 내리며
    height:50px 아이템을 찾아 브랜드 정보를 수집한다.
    반환: 브랜드 리스트 (딕셔너리들)
    """
    brands = []
    collected_urls = set()

    scroll_step = 300
    scroll_position = 0
    previous_count = -1
    stable_rounds = 0
    max_stable_rounds = 6  # 변화 없으면 종료

    # 안전을 위해 스크롤을 맨 위로
    scroll_to_top(driver, container)

    while stable_rounds < max_stable_rounds:
        # 현재 보이는 아이템들 가져오기
        try:
            if container:
                items = container.find_elements(By.CSS_SELECTOR, '[class*="BrandMenu__VirtualItem"], [class*="UIBrandItem__Wrapper"], [class*="UIBrandItem__Item"]')
            else:
                items = driver.find_elements(By.CSS_SELECTOR, '[class*="BrandMenu__VirtualItem"], [class*="UIBrandItem__Wrapper"], [class*="UIBrandItem__Item"]')
        except Exception:
            items = []

        for item in items:
            try:
                # style에서 height:50px 검사 (style이 없거나 다르면 skip)
                style = item.get_attribute('style') or ""
                if 'height: 50px' not in style:
                    # 어떤 경우엔 wrapper에 style이 없고 내부 div에 style이 있을 수 있음 -> 검사해본다
                    try:
                        inner = item.find_element(By.XPATH, './/*[contains(@style,"height: 50px")]')
                        if inner:
                            pass
                    except:
                        continue

                # URL 추출: a[href*="/brand/"]
                url = ""
                try:
                    a = item.find_element(By.CSS_SELECTOR, 'a[href*="/brand/"]')
                    url = a.get_attribute('href')
                except:
                    try:
                        url = item.get_attribute('href') or ""
                    except:
                        url = ""

                if not url:
                    continue
                if url in collected_urls:
                    continue

                # 국문명 추출 (여러 셀렉터 대비, 줄바꿈 제거)
                korean = ""
                try:
                    kn = item.find_element(By.CSS_SELECTOR, '[class*="UIBrandItem__Name-sc"], [class*="UIBrandItem__Name"], .text-body_13px_med')
                    korean = kn.text.strip().replace("\n", " ")
                except:
                    # fallback: a 태그 텍스트
                    try:
                        korean = a.text.split("\n")[0].strip()
                    except:
                        korean = ""

                # 영문명 추출
                english = ""
                try:
                    en = item.find_element(By.CSS_SELECTOR, '[class*="UIBrandItem__EnglishName"], .UIBrandItem__EnglishName-sc')
                    english = en.text.strip().replace("\n", " ")
                except:
                    # fallback: url slug
                    try:
                        english = url.rstrip('/').split('/')[-1]
                    except:
                        english = ""

                # 실제 카테고리: data-applied-tab 우선 (item 또는 내부 요소)
                actual_cat = category_name
                try:
                    brand_ele = item.find_element(By.CSS_SELECTOR, "[class*='UIBrandItem__Item-sc']")
                    applied = brand_ele.get_attribute('data-applied-tab') or ""
                    if not applied:
                        # 내부 element에 있을 수 있음
                        try:
                            inner_with_attr = item.find_element(By.XPATH, './/*[@data-applied-tab]')
                            applied = inner_with_attr.get_attribute('data-applied-tab') or ""
                        except:
                            applied = ""
                    if applied and '|' in applied:
                        actual_cat = applied.split('|', 1)[1].strip()
                except:
                    pass

                # append
                brands.append({
                    'category': actual_cat,
                    'korean_name': korean,
                    'english_name': english,
                    'url': url
                })
                collected_urls.add(url)
                
            except StaleElementReferenceException:
                continue
            except Exception:
                continue

        # 진행 로그
        print(f"  [{category_name}] 현재 수집된 브랜드 수: {len(brands)}")

        # 안정화 체크
        if len(brands) == previous_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
            previous_count = len(brands)

        # 스크롤 더 내리기
        scroll_position += scroll_step
        try:
            if container:
                driver.execute_script("arguments[0].scrollTop = arguments[1];", container, scroll_position)
            else:
                driver.execute_script("window.scrollTo(0, arguments[0]);", scroll_position)
        except:
            pass

        # 로딩 대기 (사이트 로딩 환경에 따라 늘려도 됨)
        time.sleep(0.6)

    # 수집 완료 시 스크롤 맨 위로 복구
    scroll_to_top(driver, container)
    print(f"✓ [{category_name}] 수집 완료: 총 {len(brands)}개")
    return brands

def save_to_csv(all_brands, filename=OUTPUT_CSV):
    """CSV 저장 (utf-8-sig)"""
    with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['카테고리', '국문명', '영문명', 'URL'])
        for b in all_brands:
            # 줄바꿈이나 쉼표가 들어가도 CSV가 깨지지 않게 문자열로
            writer.writerow([
                b.get('category', '').replace('\n', ' ').strip(),
                b.get('korean_name', '').replace('\n', ' ').strip(),
                b.get('english_name', '').replace('\n', ' ').strip(),
                b.get('url', '')
            ])
    print(f"✓ CSV 저장 완료: {filename}")

def main():
    driver = setup_driver(headless=False)  # 필요하면 True로 바꿔
    try:
        print(f"페이지 접속: {MAIN_URL}")
        driver.get(MAIN_URL)
        time.sleep(2.5)

        # 초반 팝업 닫기
        close_popups(driver)

        # 의류 탭(브랜드 메뉴) 선택 - 기본으로 보이는 상태라면 무시 가능
        try:
            clothing_xpath = "//div[contains(@class,'CategoryMenu__CategoryButton')]//span[contains(text(),'의류')]"
            clothing = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.XPATH, clothing_xpath)))
            try:
                clothing.click()
                time.sleep(1.0)
            except:
                pass
        except:
            pass

        # 브랜드 리스트가 들어있는 스크롤 컨테이너 찾기 (한 번만)
        container = find_scroll_container(driver)
        if container:
            print("스크롤 컨테이너를 찾음.")
        else:
            print("스크롤 컨테이너를 찾지 못함. 전체 페이지 스크롤로 시도.")

        all_brands = []
        seen_urls = set()

        for idx, cat in enumerate(CATEGORIES, 1):
            print("\n" + "="*60)
            print(f"진행 {idx}/{len(CATEGORIES)} - 카테고리: {cat}")
            print("="*60)

            # 첫 카테고리(영캐주얼)는 이미 선택된 경우가 있으므로 건너뛸 수 있음
            if idx == 1:
                # 그래도 클릭해서 보장
                click_category_tab(driver, cat)
            else:
                ok = click_category_tab(driver, cat)
                if not ok:
                    print(f"'{cat}' 탭 클릭 실패. 다음 카테고리로 이동.")
                    continue

            # 수집 시작
            time.sleep(0.8)
            brands = collect_brands_from_category(driver, cat, container=container)

            # 중복 URL 검사 후 종합 리스트에 추가
            for b in brands:
                if b['url'] not in seen_urls:
                    all_brands.append(b)
                    seen_urls.add(b['url'])

            print(f"✓ {cat} 수집 후 전체 누적 개수: {len(all_brands)}")

        # 저장
        save_to_csv(all_brands, OUTPUT_CSV)

    finally:
        driver.quit()
        print("브라우저 종료")

if __name__ == "__main__":
    main()
